In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.utils import shuffle
from PIL import ImageFont
import datetime

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import tensorflow as tf
%load_ext tensorboard

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
!rm -rf ./logs/mcc
!rm -rf /home2/kalp_shah/tmp/backup/mcc

In [8]:
files = ['ttbar','wmp','wpwm','zwpm','n2n2']

cs_lo_k = {
            'ttbar':988.57,
            'wmp'  :1.95*1e5,
            'wpwm' :124.31,
            'zwpm' :51.82,
            'n2n2' :1
          }

br_ratio = {
            'ttbar':0.67*(1-0.67)*2,
            'wmp'  :(1-0.67),
            'wpwm' :(1-0.67)*0.67*2,
            'zwpm' :0.7*(1-0.67),
            'n2n2' :1
          }

cs_nmg = {
         'ttbar':393.30,
         'wmp'  :7.865*1e4,
         'wpwm' :74.96,
         'zwpm' :14.28,
         'n2n2' :1
         }

cs_mg = {'ttbar':5.883,
          'wmp':111.5,
          'wpwm':0.944,
          'zwpm':0.2381,
          'n2n2':3.99*1e-4
        }

cs_pb = []
for f in files:
    cs_pb.append((cs_lo_k[f]*br_ratio[f]*cs_mg[f])/cs_nmg[f])

cs = [i*1e3 for i in cs_pb]
#k_f = [1.954,1.356,1.92,2.09,1.0]

cs_corr = {files[i] : cs[i] for i in range(len(files))}

In [9]:
cs_list = list(cs_corr.values())
cs_list

[6538.845366086956,
 91227.27272727272,
 692.2567850586979,
 199.5908264705882,
 0.399]

In [10]:
no_of_files = {'ttbar':0,
              'wmp':0,
              'wpwm':0,
              'zwpm':0,
              'n2n2':0
              }

red_merging = {'ttbar':98159,
               'wmp':96494,
               'wpwm':97633,
               'zwpm':81076,
               'n2n2':1e5
              }

In [11]:
def get_res(x):
    res = np.zeros(shape=(x.shape[0],5))
    #print(x.shape[0],5)
    for i in range(len(x)):
        #print(i.x[i])
        res[i,x[i]] = 1
    
    return res

In [12]:
df = []
for f in range(len(files)):
    con_df = []
    
    for i in range(1,53):
        try:
            con_df.append(pd.read_csv('~/neutrino/datasets/csvdata/' + files[f] + str(i) + '.csv'))
            no_of_files[files[f]] += 1
        except:
            pass
            #print("Not Here : ",files[f],i)
    
    df.append(pd.concat(con_df,ignore_index=True))
    df[-1]['type'] = f
    
    if files[f] == "n2n2":
        df[-1]['tag'] = 1
    else:
        df[-1]['tag'] = 0

In [13]:
total_events = []

for f in range(len(files)):
    total_events.append(no_of_files[files[f]] * red_merging[files[f]])
    
total_events

[4024519, 3859760, 3417155, 3567344, 3900000.0]

In [14]:
df[-1].head()

,ptl,etal,energyl,ptj,etaj,energyj,massj,mjj,rjj,rjl,met,n21_1,n21_2,n32_1,n32_2,infl,drfl,type,tag
0,562.212,-1.020360,881.172,1204.370,-1.184170,2154.110,90.8672,2064.210,0.951219,2.76494,2582.970,0.144000,0.270936,0.582819,0.635798,700.9430,1.054540,4,1
1,205.326,-1.533530,497.944,615.805,-0.541517,710.542,56.1178,1261.750,3.530520,2.27416,866.484,0.310466,0.331616,0.188172,0.403781,810.3350,0.847888,4,1
2,981.706,0.028503,982.105,422.185,-0.683478,527.591,55.2165,450.912,0.978519,2.71260,708.310,0.277913,0.383677,0.222872,0.272324,59.4255,0.239011,4,1
3,141.859,-0.879749,200.389,982.502,-1.092660,1631.620,78.4649,1520.520,0.724737,2.23595,1367.480,0.216641,0.222069,0.482801,0.555252,202.8420,3.400040,4,1
4,1572.220,0.736638,2018.430,694.004,-0.089478,699.541,62.0402,664.824,1.717360,1.09507,2080.970,0.526070,0.655751,0.709811,0.722818,717.7320,2.472730,4,1


In [15]:
dtset = pd.concat(df,ignore_index=True)
dtset = shuffle(dtset)
dtset['met'] = np.fabs(dtset['met'])

In [16]:
## Analysis Level Cuts
dtset = dtset[dtset['ptl'] >= 120.0][dtset['ptj'] >= 120.0][dtset['etaj'] <= 2.0][dtset['etaj'] >= -2.0]

In [17]:
train_len = int(0.8*len(dtset))
x_train = dtset.T[:-2].T[:train_len]
y_train = get_res(dtset['type'][:train_len].values)

x_test = dtset.T[:-2].T[train_len:]
y_test = get_res(dtset['type'][train_len:].values)

print('Shapes : ',x_train.shape,y_train.shape,x_test.shape,y_test.shape)

Shapes :  (9338860, 17) (9338860, 5) (2334716, 17) (2334716, 5)


In [18]:
# Callbacks
backup_callback = tf.keras.callbacks.experimental.BackupAndRestore(backup_dir="/home2/kalp_shah/tmp/backup/mcc")

log_dir = "/home2/kalp_shah/logs/mcc/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [30]:
class BCEMultiLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    
    def call(self, y_true, y_pred):

        y_pred_bce = tf.transpose(tf.transpose(y_pred)[-1])
        y_true_bce = tf.transpose(tf.transpose(y_true)[-1])
        
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=True,reduction = 'none')
        return bce(y_true, y_pred)

In [36]:
s = BCEMultiLoss()
s.call(np.array([[0,0,0,1],[0,0,0,1]]).T,np.array([[0.534,0.232,0.255,0.921],[0.534,0.232,0.255,0.421]]).T)

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.99537605, 0.81586015, 0.82875335, 0.4198848 ])>

In [46]:
class sig_callback(tf.keras.callbacks.Callback):
    def __init__(self):
        pass
    
    def on_epoch_end(self, epoch, logs=None):
        ns_t = logs['Ns']
        nb_t = logs['Nb']
        
        s_train = (ns_t/np.sqrt(nb_t))*(10/8)*(10/6)
        
        ns_v = logs['val_Ns']
        nb_v = logs['val_Nb']
        
        s_val = ns_v/np.sqrt(nb_v)*(10/8)*(10/6)
        print()
        print('The training significance is : ',s_train)
        print('The validation significance is : ',s_val)
        print('The luminosity required for 5 sigma (training) : ',np.square(5/s_train))
        print('The luminosity required for 5 sigma (validation) : ',np.square(5/s_val))

In [47]:
class NSignal(tf.keras.metrics.Metric):

    def __init__(self,cross_section,tot_events,name='Ns',**kwargs):
        super(NSignal, self).__init__(name=name, **kwargs)
        self.ns = self.add_weight(name='tp', initializer='zeros')
        self.cs = cross_section
        self.te = tot_events
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.argmax(y_true,axis=1),self.dtype)
        y_pred = tf.cast(tf.argmax(y_pred,axis=1),self.dtype)
        
        total = tf.equal(y_true, tf.cast(4,self.dtype))
        total = tf.cast(total, tf.bool)
        
        prediction = tf.equal(y_pred, tf.cast(4,self.dtype))
        prediction = tf.cast(prediction, tf.bool)
        
        signal = tf.logical_and(prediction,total)
        signal = tf.cast(signal,self.dtype)
        #signal = tf.multiply(signal,self.cs[-1])
        self.ns.assign_add((tf.reduce_sum(signal)*self.cs[-1])/self.te[-1])

    def result(self):
        return self.ns

In [48]:
class NBack(tf.keras.metrics.Metric):

    def __init__(self,cross_section,tot_events,name='Nb',**kwargs):
        super(NBack, self).__init__(name=name, **kwargs)
        self.nb = self.add_weight(name='tp', initializer='zeros')
        self.cs = cross_section
        self.te = tot_events
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.argmax(y_true,axis=1),self.dtype)
        y_pred = tf.cast(tf.argmax(y_pred,axis=1),self.dtype)
        
        comp_back = tf.cast(0,self.dtype)
        for i in range(len(self.cs) - 1):
            total = tf.equal(y_true, tf.cast(i,self.dtype))
            total = tf.cast(total, tf.bool)
        
            prediction = tf.equal(y_pred, tf.cast(4,self.dtype))
            prediction = tf.cast(prediction, tf.bool)
        
            back = tf.logical_and(prediction,total)
            back = tf.cast(back,self.dtype)
            #back = tf.multiply(back,self.cs[i])
        
            comp_back += (tf.reduce_sum(back)*self.cs[i])/self.te[i]

        self.nb.assign_add(tf.cast(comp_back,self.dtype))

    def result(self):
        return self.nb

In [49]:
model = Sequential()
input_shape = x_train.shape

from keras.layers.normalization.batch_normalization import BatchNormalization
model.add(Dense(10,activation = 'relu',input_dim = input_shape[1]))
model.add(BatchNormalization())
model.add(Dense(25,activation = 'relu',input_dim = 10))
model.add(BatchNormalization())
model.add(Dense(40,activation = 'relu',input_dim = 25))
model.add(BatchNormalization())
model.add(Dense(20,activation = 'relu',input_dim = 40))
model.add(BatchNormalization())
model.add(Dense(12,activation = 'relu',input_dim = 20))
model.add(BatchNormalization())
model.add(Dense(8,activation = 'relu',input_dim = 12))
model.add(BatchNormalization())
model.add(Dense(5,activation = 'softmax',input_dim = 8))

model.compile(optimizer='adam', loss=BCEMultiLoss(),metrics=['accuracy',NSignal(cs_list,total_events),NBack(cs_list,total_events)])

In [50]:
#try:
model.fit(x_train,y_train,epochs=200,batch_size=1024,validation_split=0.4,
              class_weight={0:5,1:7,2:4,3:4,4:.01},
          callbacks=[backup_callback,tensorboard_callback,sig_callback()])

#except:
#    pass

Epoch 1/200
5474/5476 [============================>.] - ETA: 0s - loss: 1.2258 - accuracy: 0.5385 - Ns: 0.1516 - Nb: 530.8671
The training significance is :  0.01370731079908584
The validation significance is :  0.013060163992278287
The luminosity required for 5 sigma (training) :  133056.3141019586
The luminosity required for 5 sigma (validation) :  146569.21037094007
5476/5476 [==============================] - 35s 6ms/step - loss: 1.2258 - accuracy: 0.5385 - Ns: 0.1516 - Nb: 531.0181 - val_loss: 0.3303 - val_accuracy: 0.5562 - val_Ns: 0.1010 - val_Nb: 259.3202
Epoch 2/200
5476/5476 [==============================] - ETA: 0s - loss: 1.1378 - accuracy: 0.5702 - Ns: 0.1520 - Nb: 417.6407
The training significance is :  0.01549526449355876
The validation significance is :  0.013286544151867216
The luminosity required for 5 sigma (training) :  104121.88483084088
The luminosity required for 5 sigma (validation) :  141617.17885074217
5476/5476 [==============================] - 32s 6ms/st

KeyboardInterrupt: 

In [37]:
%tensorboard --logdir /home2/kalp_shah/logs/mcc/ --port 8009

In [38]:
tot_pred = model.evaluate(dtset.T[:-2].T,get_res(dtset['type'].values),batch_size=1024)

11419/11419 [==============================] - 19s 2ms/step - loss: 0.4988 - accuracy: 0.3961 - Ns: 0.0720 - Nb: 1.3723


In [39]:
tot_pred[2],tot_pred[3]

(0.07204782217741013, 1.3723348379135132)

In [41]:
sig = (tot_pred[2])/np.sqrt((tot_pred[3]))*np.sqrt(3483)
sig

3.6296734167847253

In [42]:
total_pred = model.predict(dtset.T[:-2].T)

In [43]:
def get_back_ax(x):
    return x.argmax(axis=1)

In [44]:
sol = get_back_ax(total_pred)

In [45]:
sol.shape

(11692840,)

In [46]:
sol[2:50]

array([0, 1, 4, 4, 4, 3, 0, 3, 3, 0, 3, 0, 1, 1, 1, 3, 2, 3, 0, 3, 0, 0,
       2, 3, 1, 3, 2, 3, 0, 3, 1, 3, 3, 4, 1, 1, 2, 1, 0, 4, 1, 3, 0, 1,
       1, 0, 1, 0])

In [47]:
pred_set = dtset.copy()
pred_set['pred'] = sol

In [48]:
cor_pred = len(pred_set[train_len:][pred_set['pred'] == 4][pred_set['tag'] == 1]) + len(pred_set[train_len:][pred_set['pred'] != 4][pred_set['tag'] == 0])
print('The accuracy of the test set is : ',cor_pred/(len(pred_set[train_len:])))

The accuracy of the test set is :  0.7845638869598831


In [49]:
import warnings
warnings.filterwarnings('ignore')

In [50]:
print('Correctly identified signal (True Positive)     : ',len(pred_set[pred_set['pred'] == 4][pred_set['tag'] == 1]))
print('Falsely identified signal (False Positive)      : ',len(pred_set[pred_set['pred'] == 4][pred_set['tag'] == 0]))
print('Correctly identified background (True Negative) : ',len(pred_set[pred_set['pred'] != 4][pred_set['tag'] == 0]))
print('Falsely identified background (False Negative)  : ',len(pred_set[pred_set['pred'] != 4][pred_set['tag'] == 1]))

Correctly identified signal (True Positive)     :  704228
Falsely identified signal (False Positive)      :  793
Correctly identified background (True Negative) :  8472173
Falsely identified background (False Negative)  :  2515646


In [51]:
print('The amount of signal left is     :', len(pred_set[pred_set['pred'] == 4][pred_set['tag'] == 1])/len(df[-1]))
print('The amount of background left is :', len(pred_set[pred_set['pred'] == 4][pred_set['tag'] == 0])/np.sum([len(i) for i in df[:-1]]))

The amount of signal left is     : 0.21166306695464362
The amount of background left is : 7.861282174131266e-05


In [52]:
print('Thus, the rate of correct signal prediction is : ',len(pred_set[pred_set['pred'] == 4][dtset['tag'] == 1])/(len(pred_set[pred_set['pred'] == 4])))

Thus, the rate of correct signal prediction is :  0.9988752108093234


In [53]:
L = 3000

In [56]:
ns = cs_corr['n2n2']*(len(pred_set[pred_set['pred'] == 4][pred_set['tag'] == 1])/(no_of_files['n2n2']*red_merging['n2n2']))*L
print('n2n2',(len(pred_set[pred_set['pred'] == 4][pred_set['tag'] == 1])))
nb = 0

for i in range(len(files)-1):
    nb += cs_corr[files[i]]*(len(pred_set[pred_set['pred'] == 4][pred_set['type'] == i])/(no_of_files[files[f]]* red_merging[files[f]]))*L
    print(files[i],(len(pred_set[pred_set['pred'] == 4][pred_set['type'] == i])))

n2n2 704228
ttbar 21
wmp 53
wpwm 309
zwpm 410


In [57]:
print('The number of signal is :', ns)
print('The number of background is :', nb)
print('The significance is :',ns/np.sqrt(nb))

The number of signal is : 216.14382461538463
The number of background is : 4052.3852251302765
The significance is : 3.3953728396215817


In [70]:
model.save('/home2/kalp_shah/datasets/Models/s5')

INFO:tensorflow:Assets written to: /home2/kalp_shah/Datasets/Models/s5/assets
